In [1]:
# A script to export the reactions from an RMG mechanism and the database
# produces 3 files:

# make a dict entry for index

# species_dict_file = 'species_dict.pickle'
# reaction_dict_file = 'reaction_dict.pickle'

# correlated_uncertainty_file = 'correlated_uncertainty.pickle'


In [3]:
# example script to show how to unpack information in pickled files
import pickle
import numpy as np

In [232]:
import os
import copy
import itertools
import numpy as np
import scipy.stats
import rmgpy.data.thermo
import rmgpy.data.rmg
import rmgpy.chemkin
import rmgpy.exceptions

import rmgpy.tools.uncertainty
import matplotlib.pyplot as plt
%matplotlib inline


import importlib
importlib.reload(rmgpy.tools.uncertainty)

<module 'rmgpy.tools.uncertainty' from '/home/moon/rmg/RMG-Py/rmgpy/tools/uncertainty.py'>

In [255]:
# load example 

gas_chemkin = '/home/moon/uncertainty_estimator/cpox_pt/cpox_pt_20241020/chem_annotated-gas.inp'
gas_surface = '/home/moon/uncertainty_estimator/cpox_pt/cpox_pt_20241020/chem_annotated-surface.inp'
sp_dict = '/home/moon/uncertainty_estimator/cpox_pt/cpox_pt_20241020/species_dictionary.txt'
test_species_list, test_reaction_list = rmgpy.chemkin.load_chemkin_file(gas_chemkin, sp_dict, surface_path=gas_surface)

In [259]:
type(test_reaction_list[-1])

rmgpy.data.kinetics.family.TemplateReaction

In [286]:
# Load the database
database = rmgpy.data.rmg.RMGDatabase()
database.load(
    path = rmgpy.settings['database.directory'],
    thermo_libraries = ['surfaceThermoPt111', 'primaryThermoLibrary'],
    reaction_libraries = ['Surface/CPOX_Pt/Deutschmann2006_adjusted'],
#     kinetics_families = ['Surface_Abstraction'],
    kinetics_families = ['surface'],
    kinetics_depositories = ['training'],
    depository = True,
)

In [287]:
# Collect all of the training reactions

reaction_list = []
species_list = []  # build this up as you go to make sure all reactions use the same species object
for family in database.kinetics.families.keys():
#     print(family)
    training_depository = database.kinetics.families[family].get_training_depository()
    for i, entry in enumerate(training_depository.entries):
#         print(f'Reaction {i + 1}:')
#         display(training_depository.entries[entry].item)
        
        my_reaction = rmgpy.data.kinetics.family.TemplateReaction()
        my_reaction.reactants = training_depository.entries[entry].item.reactants
        my_reaction.products = training_depository.entries[entry].item.products
        my_reaction.family = family
#         my_reaction = training_depository.entries[entry].item
        
        
#         for j in range(len(my_reaction.reactants)):
#             i_thing = get_i_thing(my_reaction.reactants[j], species_list)
#             if i_thing < 0:
#                 my_reaction.reactants[j].thermo = database.thermo.get_thermo_data(my_reaction.reactants[j])
#                 species_list.append(my_reaction.reactants[j])
#             else:
#                 my_reaction.reactants[j] = species_list[i_thing]  # reuse species objects
            
# #             my_reaction.reactants[j].thermo = database.thermo.get_thermo_data(my_reaction.reactants[j])
#         for j in range(len(my_reaction.products)):
#             i_thing = get_i_thing(my_reaction.products[j], species_list)
#             if i_thing < 0:
#                 my_reaction.products[j].thermo = database.thermo.get_thermo_data(my_reaction.products[j])
#                 species_list.append(my_reaction.products[j])
#             else:
#                 my_reaction.products[j] = species_list[i_thing]  # reuse species objects
        
        
#             my_reaction.products[j].thermo = database.thermo.get_thermo_data(my_reaction.products[j])
        
        
        
#         my_reactions.append(training_depository.entries[entry].item)
        
        try:
            template_labels = database.kinetics.families[family].get_reaction_template_labels(my_reaction)
            template = database.kinetics.families[family].retrieve_template(template_labels)
            kinetics = database.kinetics.families[family].get_kinetics_for_template(template, degeneracy=my_reaction.degeneracy)[0]
            my_reaction.kinetics = kinetics
        except:
            continue
        reaction_list.append(my_reaction)
print(len(reaction_list))


44


In [288]:
species_list = []


for i in range(len(reaction_list)):
     for j in range(len(reaction_list[i].reactants)):
        i_thing = get_i_thing(reaction_list[i].reactants[j], species_list)
        if i_thing < 0:
            reaction_list[i].reactants[j].thermo = database.thermo.get_thermo_data(reaction_list[i].reactants[j])
            species_list.append(reaction_list[i].reactants[j])
        else:
            reaction_list[i].reactants[j] = species_list[i_thing]  # reuse species objects

for i in range(len(reaction_list)):
    for j in range(len(reaction_list[i].products)):
        i_thing = get_i_thing(reaction_list[i].products[j], species_list)
        if i_thing < 0:
            reaction_list[i].products[j].thermo = database.thermo.get_thermo_data(reaction_list[i].products[j])
            species_list.append(reaction_list[i].products[j])
        else:
            reaction_list[i].products[j] = species_list[i_thing]  # reuse species objects
        

In [289]:
len(species_list)

48

In [171]:
species_list[9].thermo

NASA(polynomials=[NASAPolynomial(coeffs=[4.85496,-0.00554135,3.01198e-05,-2.99226e-08,1.00503e-11,-11709.6,-9.25621], Tmin=(298,'K'), Tmax=(1000,'K')), NASAPolynomial(coeffs=[9.54139,-0.0104025,1.83777e-05,-9.66765e-09,1.71211e-12,-13447.6,-35.5638], Tmin=(1000,'K'), Tmax=(2000,'K'))], Tmin=(298,'K'), Tmax=(2000,'K'), Cp0=(0.01,'J/(mol*K)'), CpInf=(149.66,'J/(mol*K)'), label="""CH4X""", comment="""Thermo library: surfaceThermoPt111""")

In [139]:
def get_i_thing(thing, thing_list):
    for i in range(len(thing_list)):
        if thing.is_isomorphic(thing_list[i]):
            return i
    return -1

In [193]:
len(species_list)

53

In [194]:
len(include_species_list)

10

In [219]:
# for i in range(len(reaction_list)):
#     print(reaction_list[i].is_surface_reaction(), reaction_list[i])

In [290]:
# Trim the set to just surface reactions and the species reacting in them
# Per Joy's request, make sure to only include species involved in reactions
surface_only = True

reacting_species_set = set()
for i in range(len(reaction_list)):
    if surface_only and not reaction_list[i].is_surface_reaction():
        continue
    for sp in reaction_list[i].reactants + reaction_list[i].products:
        reacting_species_set.add(species_list.index(sp))
include_species_list = [species_list[i] for i in reacting_species_set]



include_reactions_set = set()
for i in range(len(reaction_list)):
    if surface_only and not reaction_list[i].is_surface_reaction():
        continue
    include_reactions_set.add(i)
include_reactions_list = [reaction_list[i] for i in include_reactions_set]



# Fill out the species dictionary

In [291]:
species_dict = {}
for i in range(len(include_species_list)):
    print(i, include_species_list[i])
    species_label = include_species_list[i].to_chemkin()
    
    if type(include_species_list[i].thermo) == rmgpy.thermo.thermodata.ThermoData:
        include_species_list[i].thermo = include_species_list[i].thermo.to_nasa(
            np.min(include_species_list[i].thermo.Tdata.value_si),
            np.max(include_species_list[i].thermo.Tdata.value_si),
            np.mean(include_species_list[i].thermo.Tdata.value_si)
        )
    
    species_entry = {
        'index': i,
        'RMG_adjacency_list': include_species_list[i].to_adjacency_list(),
        'NASA0': {
            'coeffs': include_species_list[i].thermo.polynomials[0].coeffs,
            'Tmin': include_species_list[i].thermo.polynomials[0].Tmin.value_si,
            'Tmax': include_species_list[i].thermo.polynomials[0].Tmax.value_si
        },
        'NASA1': {
            'coeffs': include_species_list[i].thermo.polynomials[1].coeffs,
            'Tmin': include_species_list[i].thermo.polynomials[1].Tmin.value_si,
            'Tmax': include_species_list[i].thermo.polynomials[1].Tmax.value_si
        },
        'comment': include_species_list[i].thermo.comment
    }
    
    species_dict[species_label] = species_entry



0 CH2X_1
1 HOX_3
2 CHX_1
3 CX_1
4 OX_1
5 HCOX_3
6 H2O*
7 CH4*
8 HCCH
9 X_3
10 H2CCH2
11 H2CO
12 H2
13 NO
14 H2O
15 CO2
16 CH4
17 CC_2X
18 CCH_2X
19 HCCH_2X
20 HCCH2_2X
21 HOCXO_1
22 NH2_X
23 NHX_2
24 HCOO*
25 CH3O2*
26 H2CCH2_2X
27 H2CO_2X
28 HCOOH*
29 H2CCH3_X
30 HCCH3_X
31 NH3_X
32 XCCH2
33 XCHCH2
34 HCCHX_vdW
35 H2CCH2X_vdW
36 H2COX_vdW
37 CH3X_4
38 COX_5
39 HCCH_2X
40 HX_3
41 NO_X
42 CO2X
43 NX
44 CH2OH*_2
45 HCO_2X
46 XCXCH2_3
47 HCOH*


In [270]:
include_species_list[8].thermo.Tdata.value_si

AttributeError: 'rmgpy.thermo.nasa.NASA' object has no attribute 'Tdata'

In [292]:
species_dict_file = 'species_dict.pickle'
with open(species_dict_file, 'wb') as f:
    pickle.dump(species_dict, f)

In [293]:
# read back in to check
with open(species_dict_file, 'rb') as f:
    my_data = pickle.load(f)


# Fill out the reaction dictionary

In [294]:
reaction_dict = {}
N = len(species_dict)
for i in range(len(include_reactions_list)):
    reaction_label = str(include_reactions_list[i])
    
    
    if type(include_reactions_list[i].kinetics) in [rmgpy.kinetics.surface.SurfaceArrheniusBEP, rmgpy.kinetics.surface.StickingCoefficientBEP]:
        kinetic_entry = {
            'A': include_reactions_list[i].kinetics.A.value,
            'n': include_reactions_list[i].kinetics.n.value,
            'alpha': include_reactions_list[i].kinetics.alpha.value,
            'E0': include_reactions_list[i].kinetics.E0.value,
            'A_units': include_reactions_list[i].kinetics.A.units,
            'E0_units': include_reactions_list[i].kinetics.E0.units
        }
    elif type(include_reactions_list[i].kinetics) in [rmgpy.kinetics.surface.SurfaceArrhenius, rmgpy.kinetics.surface.StickingCoefficient]:
        kinetic_entry = {
            'A': include_reactions_list[i].kinetics.A.value,
            'n': include_reactions_list[i].kinetics.n.value,
            'Ea': include_reactions_list[i].kinetics.Ea.value,
            'A_units': include_reactions_list[i].kinetics.A.units,
            'Ea_units': include_reactions_list[i].kinetics.Ea.units
        }
    
    reaction_entry = {
        'index': N + i,
        'parameterization': str(type(include_reactions_list[i].kinetics)),
        'kinetics': kinetic_entry,
        'comment': include_reactions_list[i].kinetics.comment
    }
    
    reaction_dict[reaction_label] = reaction_entry


In [295]:
reaction_dict_file = 'reaction_dict.pickle'
with open(reaction_dict_file, 'wb') as f:
    pickle.dump(reaction_dict, f)

In [296]:
# read back in to check
with open(reaction_dict_file, 'rb') as f:
    my_data = pickle.load(f)


In [227]:
my_data

{'CH2X_1 + HOX_3 <=> CH3X_4 + OX_1': {'index': 47,
  'parameterization': "<class 'rmgpy.kinetics.surface.SurfaceArrheniusBEP'>",
  'kinetics': {'A': 4.18e+17,
   'n': 0.0,
   'alpha': 0.37,
   'E0': 99.3,
   'A_units': 'm^2/(mol*s)',
   'E0_units': 'kJ/mol'},
  'comment': 'Estimated using template [Abstracting;*R-H] for rate rule [C=*;*OH]\nEuclidian distance = 2.23606797749979\nfamily: Surface_Abstraction'},
 'CHX_1 + HOX_3 <=> CH2X_1 + OX_1': {'index': 48,
  'parameterization': "<class 'rmgpy.kinetics.surface.SurfaceArrheniusBEP'>",
  'kinetics': {'A': 4.18e+17,
   'n': 0.0,
   'alpha': 0.37,
   'E0': 99.3,
   'A_units': 'm^2/(mol*s)',
   'E0_units': 'kJ/mol'},
  'comment': 'Estimated using template [Abstracting;*R-H] for rate rule [C#*;*OH]\nEuclidian distance = 2.23606797749979\nfamily: Surface_Abstraction'},
 'CX_1 + HOX_3 <=> CHX_1 + OX_1': {'index': 49,
  'parameterization': "<class 'rmgpy.kinetics.surface.SurfaceArrheniusBEP'>",
  'kinetics': {'A': 4.18e+17,
   'n': 0.0,
   'al

In [297]:
rmgpy.chemkin.save_species_dictionary('species_dictionary.txt', species_list)

In [298]:
rmgpy.chemkin.save_chemkin_file('my_reactions.inp', species_list, reaction_list)

In [228]:
# define an uncertainty object

In [299]:
uncertainty = rmgpy.tools.uncertainty.Uncertainty()

In [300]:
uncertainty.load_model('my_reactions.inp', 'species_dictionary.txt', transport_path=None, surface_path=None)

In [301]:
len(uncertainty.species_list)

47

In [302]:
len(uncertainty.reaction_list)

44

In [303]:
uncertainty.database = database

In [304]:
uncertainty.extract_sources_from_model()

In [305]:
uncertainty.assign_parameter_uncertainties()

In [306]:
uncertainty.thermo_input_uncertainties

[6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 1.5132745950421556,
 6.918,
 1.5132745950421556,
 1.5033296378372907,
 1.5,
 1.5,
 1.5,
 1.5033296378372907,
 1.5,
 1.5066519173319362,
 1.5099668870541498,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 1.5066519173319362,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918,
 6.918]

In [307]:
uncertainty.kinetic_input_uncertainties

[1.5362563145666295,
 1.5362563145666295,
 1.5362563145666295,
 1.5362563145666295,
 1.5362563145666295,
 1.5362563145666295,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.5362563145666295,
 1.5362563145666295,
 1.5362563145666295,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.5362563145666295,
 1.5362563145666295,
 1.5362563145666295,
 1.118033988749895,
 1.118033988749895,
 1.5362563145666295,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.5362563145666295,
 1.118033988749895,
 1.118033988749895,
 1.5362563145666295,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895,
 1.118033988749895]

In [233]:
uncertainty = rmgpy.tools.uncertainty.Uncertainty(species_list=species_list, reaction_list=reaction_list)

In [234]:
uncertainty.database = database

In [ ]:
# # Load the database
# database = rmgpy.data.rmg.RMGDatabase()
# database.load(
#     path = rmgpy.settings['database.directory'],
#     thermo_libraries = ['surfaceThermoPt111', 'primaryThermoLibrary'],
#     reaction_libraries = ['Surface/CPOX_Pt/Deutschmann2006_adjusted'],
# #     kinetics_families = ['Surface_Abstraction'],
#     kinetics_families = ['surface'],
#     kinetics_depositories = ['training'],
#     depository = True,
# )

ValueError: Reaction CH2X_1 + HOX_3 <=> CH3X_4 + OX_1 must be either a TemplateReaction, LibraryReaction, or PDepReaction object for source data to be extracted.

In [236]:
uncertainty.assign_parameter_uncertainties()

TypeError: 'NoneType' object is not subscriptable